In [1]:
import pandas as pd
import pyarrow as pa

In [2]:
# Получаем все файлы
train = pd.read_csv('train_events.csv')
video = pd.read_csv("video_info_v3.csv")
target = pd.read_csv("train_targets.csv")
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976
...,...,...,...,...,...,...,...,...,...
1759611,2024-06-30 17:08:36+03:00,Moscow,smartphone,mobile app,Android,Rutube,4230,video_162776,10026914
1759612,2024-06-30 01:20:16+03:00,Moscow,smartphone,browser,Android,Firefox Mobile,382,video_316157,10417567
1759613,2024-06-30 22:37:04+03:00,St.-Petersburg,desktop,browser,Windows,Microsoft Edge,1342,video_83304,10009094
1759614,2024-06-30 11:33:09+03:00,St.-Petersburg,smartphone,mobile app,Android,Rutube,801,video_132769,10574374


In [3]:
# Редактируем поле регион для последующего энкодинга
train['region'] = train['region'].replace(r"[’'`]", '', regex=True)

In [4]:
# Соединяем датасеты
train = train.merge(target, on='viewer_uid', how='inner')
train = train.merge(video, how='inner', on='rutube_video_id')


In [5]:
from datetime import timedelta
# Приводим данные к UTC
train['event_timestamp'] = pd.to_datetime(train['event_timestamp'])
train['event_timestamp'] = train['event_timestamp'] - timedelta(hours=3) # 6-12 утро 13-18 день 19-0 вечер 1-5 ночь


In [6]:
# Все регионы и их часовые пояса
time_zones = {
    'Tatarstan Republic': 3,
    'Bashkortostan Republic': 5,
    'Novosibirsk Oblast': 7,
    'Moscow': 3,
    'Moscow Oblast': 3,
    'Omsk Oblast': 7,
    'Chuvashia': 3,
    'Krasnoyarsk Krai': 7,
    'Chelyabinsk': 5,
    'Kamchatka': 11,
    'Nizhny Novgorod Oblast': 3,
    'Krasnodar Krai': 3,
    'Volgograd Oblast': 3,
    'Kaliningrad Oblast': 2,
    'Kuzbass': 7,
    'Stavropol Kray': 3,
    'Samara Oblast': 4,
    'Amur Oblast': 9,
    'Sverdlovsk Oblast': 5,
    'St.-Petersburg': 3,
    'Yamalo-Nenets': 5,
    'Orenburg Oblast': 5,
    'Khanty-Mansia': 5,
    'Kaluga Oblast': 3,
    'Tomsk Oblast': 7,
    'Novgorod Oblast': 3,
    'Arkhangelskaya': 3,
    'North Ossetia–Alania': 3,
    'Kursk Oblast': 3,
    "Leningradskaya Oblast'": 3,
    'Krasnoyarskiy': 7,
    'Ivanovo Oblast': 3,
    'Altay Kray': 7,
    'Kurgan Oblast': 5,
    'Kostroma Oblast': 3,
    'Bryansk Oblast': 3,
    'Dagestan': 3,
    'Lipetsk Oblast': 3,
    'Vladimir Oblast': 3,
    'Kirov Oblast': 3,
    'Khabarovsk': 9,
    'Tambov Oblast': 3,
    'Chukotka': 12,
    'Voronezh Oblast': 3,
    'Sverdlovsk': 5,
    'Tula Oblast': 3,
    'Krasnodarskiy': 3,
    'Irkutsk Oblast': 8,
    'Saratov Oblast': 4,
    'Khakasiya Republic': 7,
    'Penza': 3,
    'Perm Krai': 5,
    'Oryol oblast': 3,
    'Vladimir': 3,
    'Smolensk Oblast': 3,
    'Penza Oblast': 3,
    'Mordoviya Republic': 3,
    'Tyumen’ Oblast': 5,
    'Sakha': 9,
    'Primorye': 10,
    'Zabaykalskiy (Transbaikal) Kray': 9,
    'Vologda Oblast': 3,
    'Yaroslavl Oblast': 3,
    'Crimea': 3,
    'Rostov': 3,
    'Ryazan Oblast': 3,
    'Perm': 5,
    'Chechnya': 3,
    'Udmurtiya Republic': 3,
    'Tver Oblast': 3,
    'Buryatiya Republic': 8,
    'Belgorod Oblast': 3,
    'Kaluga': 3,
    'Astrakhan Oblast': 3,
    'Karelia': 3,
    'Murmansk': 3,
    'Adygeya Republic': 3,
    'Kemerovo Oblast': 7,
    'Mariy-El Republic': 3,
    'Kursk': 3,
    'Saratovskaya Oblast': 4,
    'Sakhalin Oblast': 10,
    'Ivanovo': 3,
    'Tyumen Oblast': 5,
    'Stavropol’ Kray': 3,
    'Voronezj': 3,
    'Karachayevo-Cherkesiya Republic': 3,
    'Kabardino-Balkariya Republic': 3,
    'Ulyanovsk': 4,
    'North Ossetia': 3,
    'Komi': 3,
    'Smolensk': 3,
    'Tver’ Oblast': 3,
    'Sebastopol City': 3,
    'Pskov Oblast': 3,
    'Tula': 3,
    'Orel Oblast': 3,
    'Jaroslavl': 3,
    'Tambov': 3,
    'Kalmykiya Republic': 3,
    'Primorskiy (Maritime) Kray': 10,
    'Altai': 7,
    'Magadan Oblast': 10,
    'Vologda': 3,
    'Tyva Republic': 7,
    'Nenets': 3,
    'Smolenskaya Oblast': 3,
    'Jewish Autonomous Oblast': 10,
    'Astrakhan': 3,
    'Ingushetiya Republic': 3,
    'Kirov': 3,
    'Transbaikal Territory': 9,
    'Omsk': 7,
    'Kaliningrad': 2,
    'Stavropol Krai': 3,
    'Arkhangelsk Oblast': 3,
    'Leningradskaya Oblast': 3
}

In [7]:
# Делаем колонку со смещением по временным зонам относительно региона пользователя
train['timezone_offset'] = train['region'].map(time_zones)

In [8]:
# Вычисляем время относительно часового пояса региона пользователя
train['event_timestamp_timezone'] = train['event_timestamp'] + pd.to_timedelta(train['timezone_offset'], unit='h')

In [9]:
# Вычисляем в какую часть дня пользователь просматривал видео
train['morning'] = 0
train['day'] = 0
train['evening'] = 0
train['night'] = 0
train.loc[train['event_timestamp_timezone'].dt.hour.between(6, 12, inclusive='left'), 'morning'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(13, 18, inclusive='left'), 'day'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(19, 23, inclusive='left'), 'evening'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(0, 5, inclusive='both'), 'night'] = 1

In [10]:
# Делаем Label Encoding категорий видео
category = video['category'].unique()
cat_dict = {c: c_id for c_id, c in enumerate(category)}
train['category'] = train['category'].map(cat_dict)

In [11]:
# Делаем Label Encoding типов устройства
devices = train['ua_device_type'].unique()
device_dict = {dev: dev_i for dev_i, dev in enumerate(devices)}
train['ua_device_type'] = train['ua_device_type'].map(device_dict)

In [12]:
# Делаем Label Encoding регионов
region_unique = train['region'].unique()
region_dict = {reg: reg_i for reg_i, reg in enumerate(region_unique)}
train['region'] = train['region'].map(region_dict)

In [13]:
# Пересчитываем время видео из секунды в миллисекунды
train['duration'] = train['duration'] / 1000
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,age,...,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night
0,2024-06-01 03:40:58+03:00,0,0,browser,Windows,Yandex Browser,1883,video_133074,10067243,20,...,16,2456.534,1009219,31,5,2024-06-01 08:40:58+03:00,1,0,0,0
1,2024-06-01 16:33:24+03:00,1,1,mobile app,Android,Rutube,512,video_362960,10245341,40,...,29,519.211,1006760,30,5,2024-06-01 21:33:24+03:00,0,0,1,0
2,2024-06-01 18:30:43+03:00,2,0,browser,Windows,Chrome,5647,video_96775,10894333,23,...,16,5518.280,1009257,31,3,2024-06-01 21:30:43+03:00,0,0,1,0
3,2024-06-01 20:03:42+03:00,3,1,mobile app,Android,Rutube,1521,video_161610,10029092,41,...,4,1522.069,1058671,42,3,2024-06-01 23:03:42+03:00,0,0,0,0
4,2024-06-01 19:48:09+03:00,3,1,mobile app,Android,Rutube,71,video_116245,10452976,38,...,12,1249.920,1020020,30,3,2024-06-01 22:48:09+03:00,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759611,2024-06-30 14:08:36+03:00,3,1,mobile app,Android,Rutube,4230,video_162776,10026914,32,...,1,4480.915,1095392,8,3,2024-06-30 17:08:36+03:00,0,1,0,0
1759612,2024-06-29 22:20:16+03:00,3,1,browser,Android,Firefox Mobile,382,video_316157,10417567,48,...,10,320.134,1033688,12,3,2024-06-30 01:20:16+03:00,0,0,0,1
1759613,2024-06-30 19:37:04+03:00,2,0,browser,Windows,Microsoft Edge,1342,video_83304,10009094,28,...,16,3125.675,1009219,31,3,2024-06-30 22:37:04+03:00,0,0,1,0
1759614,2024-06-30 08:33:09+03:00,2,1,mobile app,Android,Rutube,801,video_132769,10574374,43,...,16,2401.283,1009219,31,3,2024-06-30 11:33:09+03:00,1,0,0,0


In [14]:
# Считаем процент того, сколько по времени юзер смотрел видео
train['percent'] = train['total_watchtime'] / train['duration']

In [15]:
# Фильтрация по проценту
train = train.drop(train[train['percent'] > 1.1].index)

In [16]:
Q1 = train['total_watchtime'].quantile(0.4)
Q3 = train['total_watchtime'].quantile(0.9)

In [17]:
Q3

4766.0

In [18]:
mask = train['total_watchtime'].between(Q1, Q3)
train = train.loc[mask]

In [19]:
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,age,...,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 03:40:58+03:00,0,0,browser,Windows,Yandex Browser,1883,video_133074,10067243,20,...,2456.534,1009219,31,5,2024-06-01 08:40:58+03:00,1,0,0,0,0.766527
3,2024-06-01 20:03:42+03:00,3,1,mobile app,Android,Rutube,1521,video_161610,10029092,41,...,1522.069,1058671,42,3,2024-06-01 23:03:42+03:00,0,0,0,0,0.999298
6,2024-06-01 19:14:37+03:00,5,0,browser,Windows,Microsoft Edge,3882,video_53642,10058831,46,...,3983.960,1009257,31,3,2024-06-01 22:14:37+03:00,0,0,1,0,0.974407
7,2024-06-01 05:42:11+03:00,6,1,mobile app,Android,Rutube,2480,video_114660,10084832,26,...,2550.593,1084744,35,3,2024-06-01 08:42:11+03:00,1,0,0,0,0.972323
8,2024-06-01 08:09:51+03:00,7,1,mobile app,Android,Rutube,3140,video_425474,10439246,22,...,3148.075,1095392,8,7,2024-06-01 15:09:51+03:00,0,1,0,0,0.997435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759601,2024-06-30 05:32:04+03:00,27,1,mobile app,Android,Rutube,3061,video_333507,10194039,37,...,3224.000,1009257,31,3,2024-06-30 08:32:04+03:00,1,0,0,0,0.949442
1759606,2024-06-30 15:18:44+03:00,24,1,mobile app,Android,Rutube,2381,video_246846,10034725,21,...,3800.865,1036089,40,9,2024-07-01 00:18:44+03:00,0,0,0,1,0.626436
1759607,2024-06-30 12:58:04+03:00,82,0,browser,Windows,Chrome,1661,video_86744,10015493,33,...,2773.200,1002256,30,5,2024-06-30 17:58:04+03:00,0,1,0,0,0.598947
1759611,2024-06-30 14:08:36+03:00,3,1,mobile app,Android,Rutube,4230,video_162776,10026914,32,...,4480.915,1095392,8,3,2024-06-30 17:08:36+03:00,0,1,0,0,0.944004


In [20]:
train = train.drop(['title'], axis=1)

In [21]:
client_type = train['ua_client_type'].unique()
client_type_dict = {cl: cl_id for cl_id, cl in enumerate(client_type)}
train['ua_client_type'] = train['ua_client_type'].map(client_type_dict)

In [22]:
train = train.drop(train[train['ua_client_type']==2].index)

In [23]:
train['ua_os'] = train['ua_os'].fillna("Undefined")

In [24]:
oss = train['ua_os'].unique()
os_dict = {os: os_i for os_i, os in enumerate(oss)}
train['ua_os'] = train['ua_os'].map(os_dict)
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,age,...,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 03:40:58+03:00,0,0,0,0,Yandex Browser,1883,video_133074,10067243,20,...,2456.534,1009219,31,5,2024-06-01 08:40:58+03:00,1,0,0,0,0.766527
3,2024-06-01 20:03:42+03:00,3,1,1,1,Rutube,1521,video_161610,10029092,41,...,1522.069,1058671,42,3,2024-06-01 23:03:42+03:00,0,0,0,0,0.999298
6,2024-06-01 19:14:37+03:00,5,0,0,0,Microsoft Edge,3882,video_53642,10058831,46,...,3983.960,1009257,31,3,2024-06-01 22:14:37+03:00,0,0,1,0,0.974407
7,2024-06-01 05:42:11+03:00,6,1,1,1,Rutube,2480,video_114660,10084832,26,...,2550.593,1084744,35,3,2024-06-01 08:42:11+03:00,1,0,0,0,0.972323
8,2024-06-01 08:09:51+03:00,7,1,1,1,Rutube,3140,video_425474,10439246,22,...,3148.075,1095392,8,7,2024-06-01 15:09:51+03:00,0,1,0,0,0.997435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759601,2024-06-30 05:32:04+03:00,27,1,1,1,Rutube,3061,video_333507,10194039,37,...,3224.000,1009257,31,3,2024-06-30 08:32:04+03:00,1,0,0,0,0.949442
1759606,2024-06-30 15:18:44+03:00,24,1,1,1,Rutube,2381,video_246846,10034725,21,...,3800.865,1036089,40,9,2024-07-01 00:18:44+03:00,0,0,0,1,0.626436
1759607,2024-06-30 12:58:04+03:00,82,0,0,0,Chrome,1661,video_86744,10015493,33,...,2773.200,1002256,30,5,2024-06-30 17:58:04+03:00,0,1,0,0,0.598947
1759611,2024-06-30 14:08:36+03:00,3,1,1,1,Rutube,4230,video_162776,10026914,32,...,4480.915,1095392,8,3,2024-06-30 17:08:36+03:00,0,1,0,0,0.944004


In [25]:
client_names = train['ua_client_name'].unique()
client_name_dict = {cl_name: cl_name_id for cl_name_id, cl_name in enumerate(client_names)}
train['ua_client_name'] = train['ua_client_name'].map(client_name_dict)

In [26]:
from sklearn.cluster import KMeans
# Делаем группы устройств
kmeans = KMeans(n_clusters=10, max_iter=1000).fit(train[['ua_device_type', 'ua_client_type', 'ua_os', 'ua_client_name']])

In [27]:
train['device'] = kmeans.labels_

In [28]:
category_cols = [f"category_{i}" for i in range(40)]
label_cols = [f"label_{i}" for i in range(50)]
device_cols = [f"device_{i}" for i in range(len(train['device'].unique()))]
authors_cols = [f"author_{i}" for i in range(len(video['author_id'].unique()))]

In [29]:
%%time
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None
# Аггрегируем данные, генерируем новые фичи
aggregated = train.groupby('viewer_uid').agg({
    'event_timestamp_timezone': ['count', 'min', 'max', 'mean'],
    'region': get_mode,
    'label': get_mode,
    'category': get_mode,
    'total_watchtime': ['mean', 'max', 'min', 'sum'],
    'device': get_mode,
    'author_id': get_mode,
    'rutube_video_id': get_mode,
    'percent': ['mean', 'min', 'max', 'sum'],
    'duration': ['min', 'max', 'sum', 'mean']
})
aggregated

CPU times: total: 1min 28s
Wall time: 1min 29s


event_timestamp_timezone                            \
                              count                       min   
viewer_uid                                                      
10000001                         11 2024-06-01 11:54:32+03:00   
10000002                          3 2024-06-07 20:14:13+03:00   
10000004                          3 2024-06-16 14:47:45+03:00   
10000007                          4 2024-06-08 19:10:14+03:00   
10000012                          3 2024-06-17 14:08:33+03:00   
...                             ...                       ...   
11140666                          2 2024-06-30 21:54:30+03:00   
11140667                          1 2024-06-30 21:36:13+03:00   
11140684                          1 2024-06-30 22:45:45+03:00   
11140718                          1 2024-06-30 23:29:09+03:00   
11140804                          1 2024-07-01 00:49:44+03:00   

                                                                          \
                                 max                                mean   
viewer_uid                                                                 
10000001   2024-06-22 21:53:33+03:00 2024-06-06 16:33:56.636363776+03:00   
10000002   2024-06-08 00:05:26+03:00 2024-06-07 21:52:56.333333248+03:00   
10000004   2024-06-26 09:17:33+03:00 2024-06-21 06:23:30.999999744+03:00   
10000007   2024-06-21 05:58:03+03:00    2024-06-14 03:31:55.500000+03:00   
10000012   2024-06-17 16:12:29+03:00 2024-06-17 15:14:02.999999744+03:00   
...                              ...                                 ...   
11140666   2024-06-30 22:48:01+03:00    2024-06-30 22:21:15.500000+03:00   
11140667   2024-06-30 21:36:13+03:00           2024-06-30 21:36:13+03:00   
11140684   2024-06-30 22:45:45+03:00           2024-06-30 22:45:45+03:00   
11140718   2024-06-30 23:29:09+03:00           2024-06-30 23:29:09+03:00   
11140804   2024-07-01 00:49:44+03:00           2024-07-01 00:49:44+03:00   

             region    label category total_watchtime              ...  \
           get_mode get_mode get_mode            mean   max   min  ...   
viewer_uid                                                         ...   
10000001          3       31       16     3446.181818  4622  1548  ...   
10000002          3       32        1     1297.000000  1311  1270  ...   
10000004          3       30        1     2556.000000  2690  2361  ...   
10000007          4       11       16     2651.000000  2789  2260  ...   
10000012          1       26        1     2832.333333  3284  2192  ...   
...             ...      ...      ...             ...   ...   ...  ...   
11140666          3       11        1     2903.500000  3230  2577  ...   
11140667         13        8        2     2794.000000  2794  2794  ...   
11140684         39       31       16     4311.000000  4311  4311  ...   
11140718          3       47       16     2534.000000  2534  2534  ...   
11140804         22       44       16     3256.000000  3256  3256  ...   

           author_id rutube_video_id   percent                                \
            get_mode        get_mode      mean       min       max       sum   
viewer_uid                                                                     
10000001     1009257    video_204107  0.747531  0.356039  0.999892  8.222836   
10000002     1065557    video_148064  0.996139  0.994275  0.998969  2.988418   
10000004     1010000    video_342531  0.503358  0.499478  0.509045  1.510074   
10000007     1009223    video_120267  0.967032  0.897046  0.999123  3.868127   
10000012     1043618    video_191142  0.932787  0.905337  0.963500  2.798361   
...              ...             ...       ...       ...       ...       ...   
11140666     1043618    video_208085  0.903522  0.885864  0.921181  1.807045   
11140667     1036433     video_60937  0.783002  0.783002  0.783002  0.783002   
11140684     1009257    video_115806  0.639330  0.639330  0.639330  0.639330   
11140718     1009257     video_40997  

In [30]:
aggregated = aggregated.reset_index(drop=False)
aggregated

viewer_uid event_timestamp_timezone                            \
                                     count                       min   
0        10000001                       11 2024-06-01 11:54:32+03:00   
1        10000002                        3 2024-06-07 20:14:13+03:00   
2        10000004                        3 2024-06-16 14:47:45+03:00   
3        10000007                        4 2024-06-08 19:10:14+03:00   
4        10000012                        3 2024-06-17 14:08:33+03:00   
...           ...                      ...                       ...   
108541   11140666                        2 2024-06-30 21:54:30+03:00   
108542   11140667                        1 2024-06-30 21:36:13+03:00   
108543   11140684                        1 2024-06-30 22:45:45+03:00   
108544   11140718                        1 2024-06-30 23:29:09+03:00   
108545   11140804                        1 2024-07-01 00:49:44+03:00   

                                                                       region  \
                             max                                mean get_mode   
0      2024-06-22 21:53:33+03:00 2024-06-06 16:33:56.636363776+03:00        3   
1      2024-06-08 00:05:26+03:00 2024-06-07 21:52:56.333333248+03:00        3   
2      2024-06-26 09:17:33+03:00 2024-06-21 06:23:30.999999744+03:00        3   
3      2024-06-21 05:58:03+03:00    2024-06-14 03:31:55.500000+03:00        4   
4      2024-06-17 16:12:29+03:00 2024-06-17 15:14:02.999999744+03:00        1   
...                          ...                                 ...      ...   
108541 2024-06-30 22:48:01+03:00    2024-06-30 22:21:15.500000+03:00        3   
108542 2024-06-30 21:36:13+03:00           2024-06-30 21:36:13+03:00       13   
108543 2024-06-30 22:45:45+03:00           2024-06-30 22:45:45+03:00       39   
108544 2024-06-30 23:29:09+03:00           2024-06-30 23:29:09+03:00        3   
108545 2024-07-01 00:49:44+03:00           2024-07-01 00:49:44+03:00       22   

          label category total_watchtime        ... author_id rutube_video_id  \
       get_mode get_mode            mean   max  ...  get_mode        get_mode   
0            31       16     3446.181818  4622  ...   1009257    video_204107   
1            32        1     1297.000000  1311  ...   1065557    video_148064   
2            30        1     2556.000000  2690  ...   1010000    video_342531   
3            11       16     2651.000000  2789  ...   1009223    video_120267   
4            26        1     2832.333333  3284  ...   1043618    video_191142   
...         ...      ...             ...   ...  ...       ...             ...   
108541       11        1     2903.500000  3230  ...   1043618    video_208085   
108542        8        2     2794.000000  2794  ...   1036433     video_60937   
108543       31       16     4311.000000  4311  ...   1009257    video_115806   
108544       47       16     2534.000000  2534  ...   1009257     video_40997   
108545       44       16     3256.000000  3256  ...   1009257     video_80057   

         percent                                duration                       \
            mean       min       max       sum       min       max        sum   
0       0.747531  0.356039  0.999892  8.222836  4133.720  5917.000  50493.080   
1       0.996139  0.994275  0.998969  2.988418  1277.313  1316.353   3906.019   
2       0.503358  0.499478  0.509045  1.510074  4726.934  5363.360  15231.294   
3       0.967032  0.897046  0.999123  3.868127  2519.379  2859.689  10945.762   
4       0.932787  0.905337  0.963500  2.798361  2275.040  3627.378   9152.467   
...          ...       ...       ...       ...       ...       ...        ...   
108541  0.903522  0.885864  0.921181  1.807045  2909.025  3506.369   6415.394   
108542  0.783002  0.783002  0.783002  0.783002  3568.320  3568.320   3568.320   
108543  0.639330  0.639330  0.639330  0.639330  6743.000  6743.000   6743.000   
108544  0.910319  0.910319  0.910319  0.910319  2783.640  2783.640   27

In [31]:
aggregated.columns = ['_'.join(filter(None, col)).strip() for col in aggregated.columns]
aggregated.columns

Index(['viewer_uid', 'event_timestamp_timezone_count',
       'event_timestamp_timezone_min', 'event_timestamp_timezone_max',
       'event_timestamp_timezone_mean', 'region_get_mode', 'label_get_mode',
       'category_get_mode', 'total_watchtime_mean', 'total_watchtime_max',
       'total_watchtime_min', 'total_watchtime_sum', 'device_get_mode',
       'author_id_get_mode', 'rutube_video_id_get_mode', 'percent_mean',
       'percent_min', 'percent_max', 'percent_sum', 'duration_min',
       'duration_max', 'duration_sum', 'duration_mean'],
      dtype='object')

In [32]:
# Приводим время к нужному типу
aggregated['event_timestamp_timezone_min'] = aggregated['event_timestamp_timezone_min'].apply(lambda x: x.timestamp())
aggregated['event_timestamp_timezone_max'] = aggregated['event_timestamp_timezone_max'].apply(lambda x: x.timestamp())
aggregated['event_timestamp_timezone_mean'] = aggregated['event_timestamp_timezone_mean'].apply(lambda x: x.timestamp())
aggregated

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,author_id_get_mode,rutube_video_id_get_mode,percent_mean,percent_min,percent_max,percent_sum,duration_min,duration_max,duration_sum,duration_mean
0,10000001,11,1.717232e+09,1.719082e+09,1.717681e+09,3,31,16,3446.181818,4622,...,1009257,video_204107,0.747531,0.356039,0.999892,8.222836,4133.720,5917.000,50493.080,4590.280000
1,10000002,3,1.717780e+09,1.717794e+09,1.717786e+09,3,32,1,1297.000000,1311,...,1065557,video_148064,0.996139,0.994275,0.998969,2.988418,1277.313,1316.353,3906.019,1302.006333
2,10000004,3,1.718538e+09,1.719383e+09,1.718940e+09,3,30,1,2556.000000,2690,...,1010000,video_342531,0.503358,0.499478,0.509045,1.510074,4726.934,5363.360,15231.294,5077.098000
3,10000007,4,1.717863e+09,1.718939e+09,1.718325e+09,4,11,16,2651.000000,2789,...,1009223,video_120267,0.967032,0.897046,0.999123,3.868127,2519.379,2859.689,10945.762,2736.440500
4,10000012,3,1.718623e+09,1.718630e+09,1.718626e+09,1,26,1,2832.333333,3284,...,1043618,video_191142,0.932787,0.905337,0.963500,2.798361,2275.040,3627.378,9152.467,3050.822333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108541,11140666,2,1.719774e+09,1.719777e+09,1.719775e+09,3,11,1,2903.500000,3230,...,1043618,video_208085,0.903522,0.885864,0.921181,1.807045,2909.025,3506.369,6415.394,3207.697000
108542,11140667,1,1.719773e+09,1.719773e+09,1.719773e+09,13,8,2,2794.000000,2794,...,1036433,video_60937,0.783002,0.783002,0.783002,0.783002,3568.320,3568.320,3568.320,3568.320000
108543,11140684,1,1.719777e+09,1.719777e+09,1.719777e+09,39,31,16,4311.000000,4311,...,1009257,video_115806,0.639330,0.639330,0.639330,0.639330,6743.000,6743.000,6743.000,6743.000000
108544,11140718,1,1.719779e+09,1.719779e+09,1.719779e+09,3,47,16,2534.000000,2534,...,1009257,video_40997,0.910319,0.910319,0.910319,0.910319,2783.640,2783.640,2783.640,2783.640000


In [33]:
aggregated = aggregated.merge(target, on='viewer_uid', how='inner')

In [34]:
"""label_counts = train.pivot_table(index='viewer_uid', columns='label', aggfunc='size', fill_value=0)

for la in range(len(label_cols)):
    if la not in label_counts.columns:
        label_counts[la] = 0

label_counts = label_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(label_cols))))

label_counts.columns = ['viewer_uid'] + label_cols

label_df = label_counts"""

"label_counts = train.pivot_table(index='viewer_uid', columns='label', aggfunc='size', fill_value=0)\n\nfor la in range(len(label_cols)):\n    if la not in label_counts.columns:\n        label_counts[la] = 0\n\nlabel_counts = label_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(label_cols))))\n\nlabel_counts.columns = ['viewer_uid'] + label_cols\n\nlabel_df = label_counts"

In [35]:
"""category_counts = train.pivot_table(index='viewer_uid', columns='category', aggfunc='size', fill_value=0)

for category in range(len(category_cols)):
    if category not in category_counts.columns:
        category_counts[category] = 0

category_counts = category_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(category_cols))))

category_counts.columns = ['viewer_uid'] + category_cols

cat_df = category_counts"""

"category_counts = train.pivot_table(index='viewer_uid', columns='category', aggfunc='size', fill_value=0)\n\nfor category in range(len(category_cols)):\n    if category not in category_counts.columns:\n        category_counts[category] = 0\n\ncategory_counts = category_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(category_cols))))\n\ncategory_counts.columns = ['viewer_uid'] + category_cols\n\ncat_df = category_counts"

In [36]:
"""device_counts = train.pivot_table(index='viewer_uid', columns='device', aggfunc='size', fill_value=0)

for device in range(len(device_cols)):
    if device not in device_counts.columns:
        device_counts[device] = 0

device_counts = device_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(device_cols))))

device_counts.columns = ['viewer_uid'] + device_cols

device_df = device_counts"""

"device_counts = train.pivot_table(index='viewer_uid', columns='device', aggfunc='size', fill_value=0)\n\nfor device in range(len(device_cols)):\n    if device not in device_counts.columns:\n        device_counts[device] = 0\n\ndevice_counts = device_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(device_cols))))\n\ndevice_counts.columns = ['viewer_uid'] + device_cols\n\ndevice_df = device_counts"

In [37]:
t = pa.Table.from_pandas(train)

In [38]:
# Считаем кол-во просмотров по части дня
time_sum = t.select(['viewer_uid', 'morning', 'day','evening', 'night']).group_by(['viewer_uid']).aggregate([('morning', "sum"), ('day', "sum"), ('evening', "sum"), ('night', "sum")])
time_sum = time_sum.to_pandas()

In [39]:
#aggregated = aggregated.merge(cat_df, how='inner', on='viewer_uid')

In [40]:
#aggregated = aggregated.merge(label_df, how='inner', on='viewer_uid')

In [41]:
#aggregated = aggregated.merge(device_df, how='inner', on='viewer_uid')

In [42]:
aggregated = aggregated.merge(time_sum, how='inner', on='viewer_uid')

In [43]:
aggregated

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,duration_max,duration_sum,duration_mean,age,sex,age_class,morning_sum,day_sum,evening_sum,night_sum
0,10000001,11,1.717232e+09,1.719082e+09,1.717681e+09,3,31,16,3446.181818,4622,...,5917.000,50493.080,4590.280000,40,female,2,1,0,9,1
1,10000002,3,1.717780e+09,1.717794e+09,1.717786e+09,3,32,1,1297.000000,1311,...,1316.353,3906.019,1302.006333,44,male,3,0,0,2,1
2,10000004,3,1.718538e+09,1.719383e+09,1.718940e+09,3,30,1,2556.000000,2690,...,5363.360,15231.294,5077.098000,36,male,2,1,1,1,0
3,10000007,4,1.717863e+09,1.718939e+09,1.718325e+09,4,11,16,2651.000000,2789,...,2859.689,10945.762,2736.440500,40,male,2,2,0,1,1
4,10000012,3,1.718623e+09,1.718630e+09,1.718626e+09,1,26,1,2832.333333,3284,...,3627.378,9152.467,3050.822333,31,male,2,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108541,11140666,2,1.719774e+09,1.719777e+09,1.719775e+09,3,11,1,2903.500000,3230,...,3506.369,6415.394,3207.697000,37,male,2,0,0,2,0
108542,11140667,1,1.719773e+09,1.719773e+09,1.719773e+09,13,8,2,2794.000000,2794,...,3568.320,3568.320,3568.320000,24,female,1,0,0,1,0
108543,11140684,1,1.719777e+09,1.719777e+09,1.719777e+09,39,31,16,4311.000000,4311,...,6743.000,6743.000,6743.000000,47,female,3,0,0,1,0
108544,11140718,1,1.719779e+09,1.719779e+09,1.719779e+09,3,47,16,2534.000000,2534,...,2783.640,2783.640,2783.640000,37,male,2,0,0,0,0


In [44]:
# Label Encoding по видео
videos = video['rutube_video_id'].unique()
video_dict = {v: v_id for v_id, v in enumerate(videos)}

In [45]:
aggregated['sex'] = aggregated['sex'].map({'male': 1, 'female': 0})
aggregated

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,duration_max,duration_sum,duration_mean,age,sex,age_class,morning_sum,day_sum,evening_sum,night_sum
0,10000001,11,1.717232e+09,1.719082e+09,1.717681e+09,3,31,16,3446.181818,4622,...,5917.000,50493.080,4590.280000,40,0,2,1,0,9,1
1,10000002,3,1.717780e+09,1.717794e+09,1.717786e+09,3,32,1,1297.000000,1311,...,1316.353,3906.019,1302.006333,44,1,3,0,0,2,1
2,10000004,3,1.718538e+09,1.719383e+09,1.718940e+09,3,30,1,2556.000000,2690,...,5363.360,15231.294,5077.098000,36,1,2,1,1,1,0
3,10000007,4,1.717863e+09,1.718939e+09,1.718325e+09,4,11,16,2651.000000,2789,...,2859.689,10945.762,2736.440500,40,1,2,2,0,1,1
4,10000012,3,1.718623e+09,1.718630e+09,1.718626e+09,1,26,1,2832.333333,3284,...,3627.378,9152.467,3050.822333,31,1,2,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108541,11140666,2,1.719774e+09,1.719777e+09,1.719775e+09,3,11,1,2903.500000,3230,...,3506.369,6415.394,3207.697000,37,1,2,0,0,2,0
108542,11140667,1,1.719773e+09,1.719773e+09,1.719773e+09,13,8,2,2794.000000,2794,...,3568.320,3568.320,3568.320000,24,0,1,0,0,1,0
108543,11140684,1,1.719777e+09,1.719777e+09,1.719777e+09,39,31,16,4311.000000,4311,...,6743.000,6743.000,6743.000000,47,0,3,0,0,1,0
108544,11140718,1,1.719779e+09,1.719779e+09,1.719779e+09,3,47,16,2534.000000,2534,...,2783.640,2783.640,2783.640000,37,1,2,0,0,0,0


In [46]:
aggregated['rutube_video_id_get_mode'] =  aggregated['rutube_video_id_get_mode'].map(video_dict)

In [47]:
from sklearn.ensemble import IsolationForest
# Находим выбросы
def detect_outliers_isolation_forest(df, columns, contamination=0.4):
    iso = IsolationForest(contamination=contamination, random_state=42)
    preds = iso.fit_predict(df[columns])
    outliers = df[preds == -1]
    return outliers

columns = [
    'event_timestamp_timezone_count', 'event_timestamp_timezone_min', 'event_timestamp_timezone_max',
    'event_timestamp_timezone_mean', 'region_get_mode', 'label_get_mode',
    'category_get_mode', 'total_watchtime_mean', 'total_watchtime_max', "total_watchtime_min", "total_watchtime_sum",
    'rutube_video_id_get_mode', 'device_get_mode', 'author_id_get_mode', 'percent_mean', 'percent_min', "percent_max", "percent_sum", 'duration_min', 'duration_max',
    'duration_sum', 'duration_mean', 'sex', 'age_class', 'morning_sum', 'day_sum', 'evening_sum', 'night_sum']
outliers_iso = detect_outliers_isolation_forest(aggregated, columns)
print(f"Количество выбросов с Isolation Forest: {outliers_iso.shape[0]}")

Количество выбросов с Isolation Forest: 43418


In [48]:
# Удаляем выбросы
aggregated = aggregated[~aggregated["viewer_uid"].isin(outliers_iso["viewer_uid"])]

In [54]:
aggregated.columns

Index(['viewer_uid', 'event_timestamp_timezone_count',
       'event_timestamp_timezone_min', 'event_timestamp_timezone_max',
       'event_timestamp_timezone_mean', 'region_get_mode', 'label_get_mode',
       'category_get_mode', 'total_watchtime_mean', 'total_watchtime_max',
       'total_watchtime_min', 'total_watchtime_sum', 'device_get_mode',
       'author_id_get_mode', 'rutube_video_id_get_mode', 'percent_mean',
       'percent_min', 'percent_max', 'percent_sum', 'duration_min',
       'duration_max', 'duration_sum', 'duration_mean', 'age', 'sex',
       'age_class', 'morning_sum', 'day_sum', 'evening_sum', 'night_sum'],
      dtype='object')

In [49]:
aggregated.corr()

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,duration_max,duration_sum,duration_mean,age,sex,age_class,morning_sum,day_sum,evening_sum,night_sum
viewer_uid,1.000000,-0.053327,0.071722,-0.031954,0.024067,-0.015625,-0.071046,-0.162804,-0.043939,-0.069657,...,-0.086799,-0.081885,-0.070074,-0.002109,0.123248,-0.006768,-0.056014,-0.024884,-0.027087,-0.017464
event_timestamp_timezone_count,-0.053327,1.000000,-0.395736,0.438952,0.019032,0.000587,-0.024736,0.069059,-0.029893,0.350345,...,0.194583,0.899324,-0.207997,-0.051384,-0.172127,-0.045531,0.591480,0.675756,0.630926,0.390044
event_timestamp_timezone_min,0.071722,-0.395736,1.000000,0.176682,0.742186,-0.015212,-0.039842,-0.093320,-0.081628,-0.312112,...,-0.212055,-0.406480,0.022493,0.036731,0.112863,0.034703,-0.255348,-0.248313,-0.247067,-0.161967
event_timestamp_timezone_max,-0.031954,0.438952,0.176682,1.000000,0.724736,0.004156,-0.069487,-0.000826,-0.039385,0.213083,...,0.179971,0.426694,-0.052775,0.027606,-0.018739,0.027068,0.261311,0.292667,0.288996,0.162679
event_timestamp_timezone_mean,0.024067,0.019032,0.742186,0.724736,1.000000,-0.006550,-0.069718,-0.060465,-0.080265,-0.072440,...,-0.031654,0.001091,-0.023430,0.044599,0.061518,0.043107,0.000313,0.025345,0.018966,-0.005955
region_get_mode,-0.015625,0.000587,-0.015212,0.004156,-0.006550,1.000000,0.030847,0.055900,0.013018,0.016819,...,-0.009567,-0.002663,-0.005688,-0.047090,-0.053366,-0.044887,0.003141,0.007193,-0.003915,-0.019275
label_get_mode,-0.071046,-0.024736,-0.039842,-0.069487,-0.069718,0.030847,1.000000,0.321245,0.122095,0.068355,...,0.094055,0.026864,0.181571,-0.054176,-0.140008,-0.051867,0.018048,-0.014471,-0.035582,-0.026245
category_get_mode,-0.162804,0.069059,-0.093320,-0.000826,-0.060465,0.055900,0.321245,1.000000,0.098072,0.069503,...,0.084975,0.110368,0.141087,-0.089008,-0.288155,-0.088445,0.084454,0.041164,0.019256,0.018757
total_watchtime_mean,-0.043939,-0.029893,-0.081628,-0.039385,-0.080265,0.013018,0.122095,0.098072,1.000000,0.737127,...,0.257111,0.105309,0.356441,-0.065148,-0.119323,-0.059989,-0.026496,-0.031209,-0.002005,0.004218
total_watchtime_max,-0.069657,0.350345,-0.312112,0.213083,-0.072440,0.016819,0.068355,0.069503,0.737127,1.000000,...,0.473262,0.504642,0.312166,-0.064617,-0.140366,-0.058295,0.204635,0.211822,0.238659,0.162748


In [50]:
aggregated.describe()

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,duration_max,duration_sum,duration_mean,age,sex,age_class,morning_sum,day_sum,evening_sum,night_sum
count,6.512800e+04,65128.000000,6.512800e+04,6.512800e+04,6.512800e+04,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,...,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000
mean,1.017516e+07,5.292086,1.717960e+09,1.718947e+09,1.718447e+09,18.498418,29.592111,12.991831,2659.782967,3424.692866,...,5583.032341,22001.304467,4428.827708,33.295956,0.367062,1.761071,1.036589,1.335862,1.392611,0.624954
std,2.205758e+05,4.801568,6.586090e+05,6.469396e+05,5.257948e+05,20.106260,8.222832,5.866200,599.323780,911.240936,...,2055.158725,18838.223246,1438.310700,8.519873,0.482007,0.811129,1.701547,1.870570,1.872840,1.364161
min,1.000000e+07,1.000000,1.717188e+09,1.717192e+09,1.717192e+09,0.000000,0.000000,0.000000,1261.000000,1261.000000,...,1272.811000,1272.811000,1272.811000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.004066e+07,2.000000,1.717399e+09,1.718528e+09,1.718058e+09,3.000000,30.000000,13.000000,2255.500000,2681.000000,...,4022.000000,7867.543500,3184.556568,26.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.008817e+07,4.000000,1.717778e+09,1.719084e+09,1.718434e+09,11.000000,31.000000,16.000000,2621.414286,3499.000000,...,5642.240000,16250.280000,4378.560000,33.000000,0.000000,2.000000,0.000000,1.000000,1.000000,0.000000
75%,1.020713e+07,7.000000,1.718432e+09,1.719504e+09,1.718799e+09,32.000000,31.000000,16.000000,3047.357143,4252.000000,...,6780.760000,30222.845000,5536.373333,39.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000
max,1.114000e+07,36.000000,1.719798e+09,1.719805e+09,1.719800e+09,111.000000,49.000000,39.000000,4693.000000,4766.000000,...,21062.961000,140980.080000,9890.493333,54.000000,1.000000,3.000000,20.000000,25.000000,18.000000,18.000000


In [51]:
#aggregated.to_csv('aggregated.csv', index=False)

In [55]:
%%time
from imblearn.over_sampling import SMOTE
# Балансируем данные по полу
smote = SMOTE(random_state=42)

x_res, y_res = smote.fit_resample(aggregated.drop(['sex', 'age', 'age_class', 'viewer_uid'], axis=1), aggregated['sex'])

CPU times: total: 3.77 s
Wall time: 637 ms


In [56]:
from sklearn.model_selection import train_test_split
# Делим на выборки
x_train, x_test, y_train, y_test = train_test_split(x_res,
                                                    y_res, test_size=0.33, random_state=42)

In [57]:
from catboost import CatBoostClassifier
# Модель классификации
model = CatBoostClassifier(iterations=1500, eval_metric='TotalF1', learning_rate=0.25, max_depth=6,
                           early_stopping_rounds=250, task_type="GPU")

In [58]:
model.fit(x_train, y_train, eval_set=(x_test, y_test),cat_features=['region_get_mode', 'author_id_get_mode', 'rutube_video_id_get_mode'], use_best_model=True)

0:	learn: 0.8004415	test: 0.7963874	best: 0.7963874 (0)	total: 67.8ms	remaining: 1m 41s
1:	learn: 0.8004969	test: 0.7970156	best: 0.7970156 (1)	total: 94.4ms	remaining: 1m 10s
2:	learn: 0.8003759	test: 0.7969762	best: 0.7970156 (1)	total: 121ms	remaining: 1m
3:	learn: 0.8027759	test: 0.7993345	best: 0.7993345 (3)	total: 147ms	remaining: 55.1s
4:	learn: 0.8054131	test: 0.8012187	best: 0.8012187 (4)	total: 174ms	remaining: 52s
5:	learn: 0.8084864	test: 0.8061518	best: 0.8061518 (5)	total: 200ms	remaining: 49.9s
6:	learn: 0.8098605	test: 0.8072477	best: 0.8072477 (6)	total: 228ms	remaining: 48.6s
7:	learn: 0.8116227	test: 0.8090890	best: 0.8090890 (7)	total: 253ms	remaining: 47.2s
8:	learn: 0.8136479	test: 0.8118972	best: 0.8118972 (8)	total: 280ms	remaining: 46.3s
9:	learn: 0.8182726	test: 0.8165773	best: 0.8165773 (9)	total: 306ms	remaining: 45.6s
10:	learn: 0.8192739	test: 0.8183174	best: 0.8183174 (10)	total: 331ms	remaining: 44.8s
11:	learn: 0.8192927	test: 0.8181684	best: 0.8183174 

In [59]:
model.save_model('sex_model')

In [213]:
y_pred = model.predict(x_test)

In [214]:
from sklearn.metrics import f1_score, accuracy_score

acc = f1_score(y_test, y_pred, average='weighted')
acc

0.8271057964022422

In [215]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,author_id_get_mode,35.000963
1,rutube_video_id_get_mode,7.097753
2,region_get_mode,4.325385
3,duration_min,4.240913
4,device_get_mode,3.950040
5,category_get_mode,3.880949
6,label_get_mode,3.733173
7,percent_max,3.522933
8,duration_max,3.179471
9,event_timestamp_timezone_min,2.604940


In [60]:
%%time
from imblearn.over_sampling import SMOTE
# Балансируем данные по возрастному бакету
smote = SMOTE(random_state=42)

x_res, y_res = smote.fit_resample(aggregated.drop(['sex', 'age', 'age_class', 'viewer_uid'], axis=1), aggregated['age_class'])

CPU times: total: 5.59 s
Wall time: 1.22 s


In [61]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_res,
                                                    y_res, test_size=0.33, random_state=42)

In [62]:
from catboost import CatBoostClassifier
# Модель классификации
model = CatBoostClassifier(iterations=2500, eval_metric='TotalF1', learning_rate=0.1, max_depth=6,
                           early_stopping_rounds=250,  task_type="GPU", leaf_estimation_backtracking='AnyImprovement', loss_function='MultiClass')

In [63]:
model.fit(x_train, y_train, eval_set=(x_test, y_test), use_best_model=True) # cat_features=['region_get_mode', 'author_id_get_mode', 'rutube_video_id_get_mode']

0:	learn: 0.3996201	test: 0.3975512	best: 0.3975512 (0)	total: 134ms	remaining: 5m 35s
1:	learn: 0.4017797	test: 0.3986604	best: 0.3986604 (1)	total: 188ms	remaining: 3m 54s
2:	learn: 0.3947637	test: 0.3896670	best: 0.3986604 (1)	total: 223ms	remaining: 3m 5s
3:	learn: 0.3961178	test: 0.3939483	best: 0.3986604 (1)	total: 238ms	remaining: 2m 28s
4:	learn: 0.4097447	test: 0.4053701	best: 0.4053701 (4)	total: 252ms	remaining: 2m 5s
5:	learn: 0.4170515	test: 0.4138285	best: 0.4138285 (5)	total: 269ms	remaining: 1m 51s
6:	learn: 0.4198999	test: 0.4172416	best: 0.4172416 (6)	total: 282ms	remaining: 1m 40s
7:	learn: 0.4220434	test: 0.4182358	best: 0.4182358 (7)	total: 298ms	remaining: 1m 32s
8:	learn: 0.4222482	test: 0.4187213	best: 0.4187213 (8)	total: 312ms	remaining: 1m 26s
9:	learn: 0.4256901	test: 0.4235103	best: 0.4235103 (9)	total: 326ms	remaining: 1m 21s
10:	learn: 0.4313455	test: 0.4288672	best: 0.4288672 (10)	total: 338ms	remaining: 1m 16s
11:	learn: 0.4359400	test: 0.4337001	best: 

In [220]:
y_pred = model.predict(x_test)

In [221]:
f = f1_score(y_test, y_pred, average='weighted')
f

0.5899754726468687

In [222]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,author_id_get_mode,11.354947
1,rutube_video_id_get_mode,6.091383
2,label_get_mode,5.937290
3,event_timestamp_timezone_max,5.287377
4,duration_min,5.018028
5,event_timestamp_timezone_min,4.968720
6,region_get_mode,4.917844
7,event_timestamp_timezone_mean,4.755795
8,category_get_mode,4.728948
9,duration_max,4.558634


In [66]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(aggregated.drop(['sex', 'age', 'age_class', 'viewer_uid'], axis=1),
                                                    aggregated['age'], test_size=0.33, random_state=42)

In [67]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=2500, eval_metric='MAE', learning_rate=0.1, max_depth=6,
                           early_stopping_rounds=250,  task_type="GPU", leaf_estimation_backtracking='AnyImprovement')
model.fit(x_train, y_train, eval_set=(x_test, y_test),cat_features=['region_get_mode', 'author_id_get_mode', 'rutube_video_id_get_mode'], use_best_model=True)

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 6.9380737	test: 6.8959152	best: 6.8959152 (0)	total: 62.9ms	remaining: 2m 37s
1:	total: 101ms	remaining: 2m 5s
2:	total: 137ms	remaining: 1m 54s
3:	total: 169ms	remaining: 1m 45s
4:	total: 203ms	remaining: 1m 41s
5:	learn: 6.6380178	test: 6.5858426	best: 6.5858426 (5)	total: 237ms	remaining: 1m 38s
6:	total: 277ms	remaining: 1m 38s
7:	total: 314ms	remaining: 1m 37s
8:	total: 349ms	remaining: 1m 36s
9:	total: 381ms	remaining: 1m 34s
10:	learn: 6.5005629	test: 6.4394803	best: 6.4394803 (10)	total: 412ms	remaining: 1m 33s
11:	total: 443ms	remaining: 1m 31s
12:	total: 481ms	remaining: 1m 31s
13:	total: 518ms	remaining: 1m 31s
14:	total: 554ms	remaining: 1m 31s
15:	learn: 6.4201372	test: 6.3535621	best: 6.3535621 (15)	total: 590ms	remaining: 1m 31s
16:	total: 620ms	remaining: 1m 30s
17:	total: 651ms	remaining: 1m 29s
18:	total: 679ms	remaining: 1m 28s
19:	total: 711ms	remaining: 1m 28s
20:	learn: 6.3786560	test: 6.3085532	best: 6.3085532 (20)	total: 749ms	remaining: 1m 28s
21:	tot

In [223]:
0.7 * f + 0.3 * acc

0.6611145697734807

In [64]:
model.save_model("age_model")

In [68]:
model.save_model("int_age_model")